In [1]:
import pandas as pd
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot as plt
import scipy.stats as stats
from scipy.special import inv_boxcox

plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})


#Split time series into a training/test
def split_to_train_test(series, test_percent=0.50):
    split_point = int(test_percent * len(series))
    return series[:-split_point], series[-split_point:]

#Select a percentage from the time series
def data_proportion(series, percent=1.0, from_end=True):
    if from_end:
        series = series[-int(percent * len(series)):]
    else:
        series = series[:int(percent * len(series))]
    return series


In [43]:
#load data
series = read_csv('timeserie_prepro.csv', 
                  parse_dates=['data'], 
                  index_col='data',
                  squeeze=True)
print(f'Total data: {len(series)}')

Total data: 7701


In [44]:
series_per = data_proportion(series, percent=.5, from_end=True)
print(f'Percentage data: {len(series_per)}')

series_per

Percentage data: 3850


data
2010-07-18     8872.590
2010-07-19     9118.390
2010-07-20     9226.760
2010-07-21     9231.770
2010-07-22     9136.080
                ...    
2021-01-27    12308.892
2021-01-28    12550.095
2021-01-29    12413.574
2021-01-30    11988.032
2021-01-31    11540.266
Name: energia, Length: 3850, dtype: float64

In [45]:
series_per_bc , series_per_lambda = stats.boxcox(series_per) 

In [46]:
for i in range(series_per.size):
     series_per[i] = series_per_bc[i]

In [47]:
series_per

data
2010-07-18    43128.828108
2010-07-19    44558.520751
2010-07-20    45191.236878
2010-07-21    45220.522565
2010-07-22    44661.704634
                  ...     
2021-01-27    63743.028843
2021-01-28    65236.517163
2021-01-29    64390.513501
2021-01-30    61765.089185
2021-01-31    59021.957875
Name: energia, Length: 3850, dtype: float64

In [48]:

#split data
train_bc, test_bc = split_to_train_test(new_series, test_percent=0.40)
print('Training %d, Teste %d' % (len(train_bc), len(test_bc)))

Training 2310, Teste 1540


In [49]:
# Prepare data
test_bc = test_bc.values
train_bc = [x for x in train_bc]

# Walk-forward validation
predictions = list()

for i in range(len(test_bc)):
    # Predict
    yhat = train_bc[-1]
    # Store forecast in list of predictions
    predictions.append(yhat)
    # Add actual observation to train for the next loop
    obs = test_bc[i]
    train_bc.append(obs)
    #print('>Predicted=%.3f, Expected=%.3f' % (yhat, obs))